# Set up

In [1]:
import tensorflow as tf
from pathlib import Path

2024-03-28 11:14:04.789773: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Generating Shakespearean Text Using a Character RNN 

## Creating the Training Dataset

In [2]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url, cache_dir=".")
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
# Extra code shows a short text sample
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [4]:
# Extra code shows all 39 distinct characters (after converting to lower case)
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [5]:
text_vec_layer = tf.keras.layers.TextVectorization(
    split="character", standardize="lower"
)
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

2024-03-28 11:14:06.676724: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

In [7]:
n_tokens

39

In [8]:
dataset_size

1115394

In [9]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [10]:
list(to_dataset(text_vec_layer(["To be"])[0], length=4))

2024-03-28 11:14:07.306267: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [5]
	 [[{{node Placeholder/_0}}]]
2024-03-28 11:14:07.306427: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [5]
	 [[{{node Placeholder/_0}}]]


[(<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 4,  5,  2, 23]])>,
  <tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 5,  2, 23,  3]])>)]

In [11]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

## Building and Training the Char-RNN Model


**Warning**: the following code may take one or two hours to run, depending on your GPU. Without GPU, it may take over 24 hours. This is why I won't bother training it, instead I will use  pretrained one from the author.

**Note**: The `GRU` class will only use cuDNN acceleration (assuming you have a GPU with CUDA) when using the default values for the following arguments: `activation`, `recurrent_activation`, `recurrent_dropout`, `dropout`, `unroll`, `use_bias` and `reset_after`. Read the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU) for more details.

```python
tf.random.set_seed(42)
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
        tf.keras.layers.GRU(128, return_sequences=True),
        tf.keras.layers.Dense(n_tokens, activation="softmax"),
    ]
)
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('my_shakespeare_model', monitor='val_accuracy', save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_checkpoint])
```

```python
shakespeare_model = tf.keras.Sequential([
    text_vec_layer, # adapted layer
    tf.keras.layers.Lambda(lambda X: X-2), # no <PAD> or <UNK> tokens
    model
])
```

In [12]:
# extra code downloads a pretrained model
url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = tf.keras.utils.get_file(
    "shakespeare_model.tgz", url, extract=True, cache_dir="."
)
model_path = Path(path).with_name("shakespeare_model")
shakespeare_model = tf.keras.models.load_model(model_path)

352865/352865 [==============================] - 1s 2us/step


2024-03-28 11:14:11.599255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2024-03-28 11:14:11.599338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2024-03-28 11:14:11.599366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [18]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_predict = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_predict + 2]

1/1 [==============================] - 0s 31ms/step


'e'